In [21]:
import swat
import os
import pandas as pd
pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt

hostValue = os.environ.get('CASHOST')
portValue = os.environ.get('CASPORT')
passwordToken=os.environ.get('SAS_VIYA_TOKEN')

conn = swat.CAS(hostname=hostValue,port=portValue,password=passwordToken)

In [22]:
lookupdata = {
    'LD':[2,2,2,2,2],
    'STORE':[100,100,100,100,100],
    'SKU':[1213,112321,4546,877486,764531],
    'MIN':[200,200,300,200,200]
}
df1 = pd.DataFrame(lookupdata)
tbl1 = conn.upload_frame(df1, 
                  casout = {
                      'name':'tbl1',
                      'caslib':'casuser',
                      'replace':True
                  })

tbl = {
    'LD':[2,2,2,3,3],
    'STORE':[100,100,100,100,100],
    'SKU':[1213,112321,4546,877486,764531],
    'VALUE':[200,200,300,200,200]    
}

df2 = pd.DataFrame(tbl)
tbl2 = conn.upload_frame(df2, 
                  casout = {
                      'name':'tbl2',
                      'caslib':'casuser',
                      'replace':True
                  })

NOTE: Cloud Analytic Services made the uploaded file available as table TBL1 in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TBL1 has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table TBL2 in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TBL2 has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


# Preview CAS tables

In [23]:
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TBL1,5,4,0,utf-8,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,UTF8,1.993995e+09,1.993995e+09,1.993995e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-03-09T10:29:44-05:00,1.993995e+09
1,TBL2,5,4,0,utf-8,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,UTF8,1.993995e+09,1.993995e+09,1.993995e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-03-09T10:29:44-05:00,1.993995e+09


In [24]:
display(tbl1.head(), tbl2.head())

,LD,STORE,SKU,MIN
0,2.0,100.0,1213.0,200.0
1,2.0,100.0,112321.0,200.0
2,2.0,100.0,4546.0,300.0
3,2.0,100.0,877486.0,200.0
4,2.0,100.0,764531.0,200.0


,LD,STORE,SKU,VALUE
0,2.0,100.0,1213.0,200.0
1,2.0,100.0,112321.0,200.0
2,2.0,100.0,4546.0,300.0
3,3.0,100.0,877486.0,200.0
4,3.0,100.0,764531.0,200.0


# Workaround

## Use SQL

In [25]:
conn.loadActionSet('fedSQL')

NOTE: Added action set 'fedSQL'.


[actionset]

 'fedSQL'

+ Elapsed: 0.00128s, user: 0.000647s, sys: 0.000613s, mem: 0.239mb

In [26]:
joindata = '''
    create table joineddata{options replace=True} as
    select summary.LD, 
           summary.STORE,
           summary.SKU, 
           summary.MIN,
           main.VALUE
        from casuser.tbl1 as summary inner join 
             casuser.tbl2 as main
        on summary.LD = main.LD and
           summary.STORE = main.STORE and
           summary.SKU = main.SKU;
'''
conn.execDirect(query = joindata)

NOTE: Table JOINEDDATA was created in caslib CASUSER(Peter.Styliadis@sas.com) with 3 rows returned.


+ Elapsed: 0.102s, user: 0.107s, sys: 0.0917s, mem: 100mb

In [27]:
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TBL1,5,4,0,utf-8,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,2023-03-09T10:29:57-05:00,UTF8,1.993995e+09,1.993995e+09,1.993995e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-03-09T10:29:44-05:00,1.993995e+09
1,TBL2,5,4,0,utf-8,2023-03-09T10:29:44-05:00,2023-03-09T10:29:44-05:00,2023-03-09T10:29:57-05:00,UTF8,1.993995e+09,1.993995e+09,1.993995e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-03-09T10:29:44-05:00,1.993995e+09
2,JOINEDDATA,3,5,0,utf-8,2023-03-09T10:29:57-05:00,2023-03-09T10:29:57-05:00,2023-03-09T10:29:57-05:00,UTF8,1.993995e+09,1.993995e+09,1.993995e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN


In [17]:
joineddata = conn.CASTable('joineddata', caslib = 'casuser')
joineddata.head()

,LD,STORE,SKU,MIN,VALUE
0,2.0,100.0,1213.0,200.0,200.0
1,2.0,100.0,112321.0,200.0,200.0
2,2.0,100.0,4546.0,300.0,300.0


In [28]:
conn.terminate()